In [2]:
import os

In [1]:
%pwd

'c:\\Users\\SLEEMTEECH\\desktop\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SLEEMTEECH\\desktop\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_devise_train_batch_size: int
    weight_decay: float
    loading_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_devise_train_batch_size= params.per_devise_train_batch_size,
            weight_decay = params.weight_decay,
            loading_steps = params.loading_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config



    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs,
        #     warmup_steps=self.config.warmup_steps,per_device_train_batch_size=self.config.per_device_train_batch_size,
        #     per_device_eval_batch_size= self.config.per_device_train_batch_size,evaluation_strategy=self.config.evaluation_strategy,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     eval_steps=self.config.eval_steps, save_steps=1e6,gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )


        trainer_args= TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1,per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,evaluation_strategy='steps',
            eval_steps=500, save_steps=1e6,gradient_accumulation_steps=16
        )

        trainer = Trainer(model=model_pegasus,args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-23 10:48:28,652: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 10:48:28,667: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-23 10:48:28,670: INFO: common: created directory at: artifacts]
[2025-01-23 10:48:28,673: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\SLEEMTEECH\anaconda3\envs\textS\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\SLEEMTEECH\AppData\Local\Temp\ipykernel_33264\1534417185.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus,args=trainer_args,
 20%|█▉        | 10/51 [21:36<1:23:28, 122.16s/it]

{'loss': 3.1101, 'grad_norm': 510.9972229003906, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [41:42<1:01:53, 119.79s/it]

{'loss': 3.0466, 'grad_norm': 253.1725616455078, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9899, 'grad_norm': 323.4247741699219, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [1:51:18<02:53, 173.27s/it]

{'loss': 2.8525, 'grad_norm': 634.4304809570312, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [1:54:16<00:00, 174.62s/it]c:\Users\SLEEMTEECH\anaconda3\envs\textS\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
100%|██████████| 51/51 [1:57:07<00:00, 137.80s/it]


{'train_runtime': 7027.8004, 'train_samples_per_second': 0.117, 'train_steps_per_second': 0.007, 'train_loss': 3.022768221649469, 'epoch': 1.0}
